# GWExPy FrequencySeries 新機能チュートリアル

`gwexpy` の `FrequencySeries` および `FrequencySeriesMatrix` は、`gwpy` の周波数領域データ処理機能を拡張し、制御工学との連携や高度な解析を可能にします。
このノートブックでは、以下の新機能を紹介します。

1. **FrequencySeriesMatrix**: 多チャンネル周波数データの効率的処理
2. **位相・群遅延解析**: `phase`, `angle`, `degree`, `group_delay`
3. **制御工学連携**: Python Control Systems Library (`control`) との相互運用

In [ ]:
import numpy as np
from astropy import units as u
from gwexpy.frequencyseries import FrequencySeries, FrequencySeriesMatrix

# 乱数シード固定
np.random.seed(42)

## 1. FrequencySeriesMatrix: 多チャンネル周波数データ

`FrequencySeriesMatrix` を使うと、多数の周波数スペクトルや伝達関数を一括管理できます。

In [ ]:
# サンプルデータ生成
n_channels = 2
n_freqs = 100
df = 1.0 * u.Hz
frequencies = np.arange(n_freqs) * df.value

# 複素数データ (振幅と位相を持つと想定)
amplitude = np.random.rand(n_channels, 1, n_freqs)
phase = np.random.uniform(-np.pi, np.pi, size=(n_channels, 1, n_freqs))
data_complex = amplitude * np.exp(1j * phase)

# FrequencySeriesMatrix 作成
fsm = FrequencySeriesMatrix(
    data_complex,
    f0=0 * u.Hz,
    df=df,
    unit="V",
    channel_names=["CH_A", "CH_B"]
)

print("FrequencySeriesMatrix Summary:")
print(fsm)
print("Shape:", fsm.shape)

## 2. 位相と群遅延の解析

`FrequencySeries` クラスには、位相 (`phase`, `angle`) や群遅延 (`group_delay`) を計算する便利なメソッドが追加されています。

In [ ]:
# 単一チャンネルを取り出す
fs = fsm[0, 0]

# 位相の計算 (ラジアン)
phi = fs.phase(unwrap=True)
print("\nUnwrapped Phase (rad) at 10Hz:", phi.value[10])

# 位相の計算 (度)
deg = fs.degree(unwrap=True)
print("Unwrapped Phase (deg) at 10Hz:", deg.value[10])

# 群遅延 (Group Delay) の計算
# Group Delay = -d(phi)/d(omega)
gd = fs.group_delay()
print("\nGroup Delay (s) at 10Hz:", gd.value[10])

## 3. Python Control Library との連携

制御工学の分野で標準的な `control` ライブラリの **Frequency Response Data (FRD)** オブジェクトと相互変換が可能です。
これにより、GWExPyで計測した伝達関数を、制御系の設計や解析に直接利用できます。

In [ ]:
try:
    import control
    
    # FrequencySeries -> control.FRD 変換
    # frequency_unit="Hz" を指定すると、内部で rad/s に適切に変換して渡します
    frd_sys = fs.to_control_frd(frequency_unit="Hz")
    
    print("\n--- Converted to Control FRD ---")
    print(frd_sys)
    
    # ボード線図のプロット (controlライブラリの機能)
    # control.bode(frd_sys) # (プロット環境があれば実行可能)
    
    # control.FRD -> FrequencySeries 復元
    fs_restored = FrequencySeries.from_control_frd(frd_sys, frequency_unit="Hz")
    
    print("\n--- Restored FrequencySeries ---")
    print(fs_restored)
    
except ImportError:
    print("Python Control Systems Library is not installed.")

## 4. 行列演算によるスペクトル解析

`FrequencySeriesMatrix` は行列演算をサポートしているため、チャンネル間での演算も容易です。

In [ ]:
# 例: 全チャンネルの振幅スペクトル（絶対値）を計算
amp_matrix = np.abs(fsm.value)
print("\nAmplitude Matrix mean:", amp_matrix.mean())

## まとめ

`FrequencySeries` の拡張により、単純なスペクトル解析だけでなく、位相特性の評価、群遅延の計算、そして制御理論への応用まで、シームレスに行えるようになりました。